In [1]:
%config IPCompleter.greedy = True
%config Completer.use_jedi = False

In [4]:
import numpy as np # linear algebra
from numpy.lib.stride_tricks import sliding_window_view
import cupy as cp
import tensorflow as tf
import tensorflow_probability as tfp
import timeseriesanalysis as tsa
!pip install bezier
!pip install lmoments3
import lmoments3 as lm
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from tqdm import tqdm
from numba import njit
import itertools 
from scipy.special import comb
from scipy import signal

  Using cached lmoments3-1.0.8-py3-none-any.whl.metadata (46 kB)
Using cached lmoments3-1.0.8-py3-none-any.whl (48 kB)


# **Kernel Function Computations For GPUs**

In [2]:
def remove_duplicate_arrays(array_list):
    """
    Removes duplicate NumPy arrays from a list, returning a new list
    containing only unique arrays.

    Args:
        array_list (list of numpy arrays): A list containing NumPy arrays.

    Returns:
        list of numpy arrays: A new list containing only the unique NumPy arrays
                              from the input list. The order of arrays in the
                              returned list is preserved from the original list
                              as much as possible (the *first* occurrence of each
                              unique array is kept).
    """

    unique_arrays = []
    seen = set()  # Use a set to track seen array's byte representations

    for arr in array_list:
        # Convert the array to bytes for efficient set comparison
        arr_bytes = arr.tobytes()

        if arr_bytes not in seen:
            unique_arrays.append(arr)
            seen.add(arr_bytes)

    return unique_arrays
    

def bezier_curve_fast_cupy(control_points, num_points=100):
    """
    Generates 2D Bézier curves for a batch of control point arrangements,
    optimized for speed using CuPy, and scales each curve so that its points sum to 1.

    Args:
        control_points (cupy array): A CuPy array of shape (num_arrangements, n+1, 2)
            representing the control points for each Bézier curve.
        num_points (int): The number of points to generate along each curve.

    Returns:
        cupy array: A CuPy array of shape (num_arrangements, num_points, 2) containing the (x, y)
            coordinates of the points on the Bézier curves, scaled to sum to 1.
    """

    n = control_points.shape[1] - 1
    num_arrangements = control_points.shape[0]

    # Precompute Bernstein basis polynomials for all t values
    t = cp.linspace(0, 1, num_points)[:, None]

    # Precompute the binomial coefficients (on the CPU, then transfer)
    binomial_coefficients = np.array([comb(n, i) for i in range(n + 1)])
    binomial_coefficients_gpu = cp.asarray(binomial_coefficients)

    # Vectorized calculation of Bernstein polynomials
    bernstein_polynomials = binomial_coefficients_gpu * (t ** cp.arange(n + 1).reshape(1, -1)) * ((1 - t) ** cp.arange(n, -1, -1).reshape(1, -1))

    # Calculate curve points using vectorized operations (Batch Matrix Multiplication)
    points = cp.matmul(bernstein_polynomials, control_points)

    # Scale the curve to sum to 1
    curve_sum = cp.sum(points, axis=1, keepdims=True)
    mask = (curve_sum != 0)
    points = cp.where(mask, points / curve_sum, points)

    return points.astype(cp.float16)



def generate_coordinate_arrangements_cupy(point_ranges):
    """
    Generates all possible coordinate arrangements for a specified number of points,
    where each point can have its own independent x and y ranges.

    Args:
        point_ranges (list of tuples): A list of tuples, where each tuple represents the
            (x_range, y_range) for a single point.  Each x_range and y_range are themselves
            tuples specifying (min_x, max_x) and (min_y, max_y), respectively.

            For example:
            `point_ranges = [((0, 2), (0, 3)), ((1, 4), (2, 5)), ((0, 1), (1, 1))]`
            means:
                - Point 1: x can be 0, 1, or 2; y can be 0, 1, 2, or 3.
                - Point 2: x can be 1, 2, 3, or 4; y can be 2, 3, 4, or 5.
                - Point 3: x can be 0 or 1; y can only be 1.

    Returns:
        A CuPy array of shape (num_arrangements, num_points, 2).
    """
    arrangements = []
    num_points = len(point_ranges)

    def generate_recursive(current_arrangement, point_index):
        if point_index == num_points:
            arrangements.append(np.array(current_arrangement))
            return

        x_range, y_range = point_ranges[point_index]
        for x in range(x_range[0], x_range[1] + 1):
            for y in range(y_range[0], y_range[1] + 1):
                new_arrangement = current_arrangement + [[x, y]]
                generate_recursive(new_arrangement, point_index + 1)

    generate_recursive([], 0)
    return cp.asarray(arrangements)


# Example Usage:
# Define the individual x and y ranges for each point

min_x = 0
max_x = 4
min_y = 0
max_y = 7

point_ranges = [
    ((min_x, min_x), (min_y, max_y)),
    ((min_x, max_x), (min_y, max_y)),
    ((min_x, max_x), (min_y, max_y)),
    ((max_x, max_x), (min_y, max_y))  
]

# Generate all arrangements
all_arrangements_gpu = generate_coordinate_arrangements_cupy(point_ranges)

min_window_size = 10
max_window_size = 1000
window_size_step = 15
loop_points = range(min_window_size, max_window_size, window_size_step)
enum_loop_points = enumerate(loop_points)

# Preallocate the array to store the kernel functions (on the CPU)
kernel_functions = [] # Changed from np.empty to a list

with tqdm(desc="Processing") as pbar:
    for window_index, window_size in enum_loop_points:
        # Calculate all Bezier curves for the current window size in one go!
        all_curves = bezier_curve_fast_cupy(all_arrangements_gpu, num_points=window_size) #shape is num_arrangements, num_points, 2
        y_coords_gpu = all_curves[:, :, 1]  # Extract y-coordinates; shape is num_arrangements, num_points

        # Copy y-coordinates to CPU as a NumPy array
        y_coords_cpu = cp.asnumpy(y_coords_gpu)  #Shape is now num_arrangements, num_points

        #Store NumPy Array as a 2D array in the list.
        kernel_functions.append(y_coords_cpu) #Changed to append, not assign

        pbar.update(1)

# leave unique kernel functions only
for i in range(len(kernel_functions)):
    kernel_functions[i] = remove_duplicate_arrays(kernel_functions[i])

Processing: 66it [00:11,  5.72it/s]


# **Kernel Function Computations For TPUs**

In [ ]:
tfd = tfp.distributions

def bezier_curve_fast_tf(control_points, num_points=100):
    """
    Generates 2D Bézier curves for a batch of control point arrangements,
    optimized for speed using TensorFlow and TPUs, and scales each curve so that its points sum to 1.

    Args:
        control_points (tf.Tensor): A TensorFlow tensor of shape (num_arrangements, n+1, 2)
            representing the control points for each Bézier curve.
        num_points (int): The number of points to generate along each curve.

    Returns:
        tf.Tensor: A TensorFlow tensor of shape (num_arrangements, num_points, 2) containing the (x, y)
            coordinates of the points on the Bézier curves, scaled to sum to 1.
    """

    n = tf.cast(tf.shape(control_points)[1] - 1, dtype=tf.float32)  # Use tf.cast
    num_arrangements = tf.shape(control_points)[0]

    # Precompute Bernstein basis polynomials for all t values
    t = tf.linspace(0.0, 1.0, num_points)[:, None]

    # Precompute the binomial coefficients (on the CPU, then transfer if needed - usually XLA handles this fine)
    binomial_coefficients = np.array([comb(int(n), i) for i in range(int(n) + 1)])
    binomial_coefficients_tf = tf.constant(binomial_coefficients, dtype=tf.float32) # Convert to tf.constant

    # Vectorized calculation of Bernstein polynomials
    arange_n_plus_1 = tf.range(int(n + 1), dtype=tf.float32)
    bernstein_polynomials = binomial_coefficients_tf * (t ** arange_n_plus_1) * ((1 - t) ** tf.reverse(arange_n_plus_1, axis=[0]))

    # Calculate curve points using vectorized operations (Batch Matrix Multiplication)
    bernstein_polynomials = tf.cast(bernstein_polynomials, dtype=tf.float32)
    control_points = tf.cast(control_points, dtype=tf.float32)
    points = tf.matmul(bernstein_polynomials, control_points)

    # Scale the curve to sum to 1
    curve_sum = tf.reduce_sum(points, axis=1, keepdims=True)
    mask = tf.not_equal(curve_sum, 0)
    points = tf.where(mask, points / curve_sum, points)

    return tf.cast(points, dtype=tf.float16)


def generate_coordinate_arrangements_tf():
    """
    Generates all possible coordinate arrangements for four points, as a TensorFlow tensor.

    Returns:
        A TensorFlow tensor of shape (num_arrangements, 4, 2).
    """
    arrangements = []
    for y1 in range(0, 7):
        for x2 in range(0, 5):
            for y2 in range(0, 7):
                for x3 in range(0, 5):
                    for y3 in range(0, 7):
                        for y4 in range(0, 7):
                            point1 = [0, y1]
                            point2 = [x2, y2]
                            point3 = [x3, y3]
                            point4 = [4, y4]
                            arrangements.append(np.array([point1, point2, point3, point4]))

    return tf.constant(np.array(arrangements), dtype=tf.float32)


# TPU Strategy Setup (REQUIRED if running on TPU)
# This part needs to be adjusted based on your specific TPU setup
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default strategy in case of no TPU

print("REPLICAS: ", strategy.num_replicas_in_sync)

# Now, wrap everything that uses the TPU inside the strategy scope
with strategy.scope():
    # Generate all arrangements
    all_arrangements_tf = generate_coordinate_arrangements_tf()

    min_window_size = 10
    max_window_size = 1000
    window_size_step = 15
    loop_points = range(min_window_size, max_window_size, window_size_step)
    enum_loop_points = enumerate(loop_points)

    # Preallocate the array to store the kernel functions (on the CPU)
    kernel_functions = np.empty((len(loop_points), len(all_arrangements_tf)), dtype=object)


    with tqdm(desc="Processing") as pbar:
        for window_index, window_size in enum_loop_points:
            # Calculate all Bezier curves for the current window size in one go!
            all_curves = bezier_curve_fast_tf(all_arrangements_tf, num_points=window_size) #shape is num_arrangements, num_points, 2
            y_coords_tf = all_curves[:, :, 1]  # Extract y-coordinates; shape is num_arrangements, num_points

            # Convert y_coords_tf to NumPy array
            y_coords_np = y_coords_tf.numpy()

            # Store NumPy Array in 2d output.
            kernel_functions[window_index, :] = [y_coords_np[i, :] for i in range(y_coords_np.shape[0])]

            pbar.update(1)

print("Finished processing.")

# Computations for Multiscale LTI Filter Information

In [4]:
def conv_subtract_scale(time_series, kernel_functions):
    """
    Performs convolution, subtraction, and scaling operations efficiently, 
    minimizing data conversions.

    Args:
        time_series: A 1D NumPy array of time series data.
        kernel_functions: A 2D NumPy array of kernel functions.

    Returns:
        A TensorFlow tensor representing the result of the operations.
    """

    # 1. Preprocessing & Reshaping in NumPy (Optimize for TF Input)
    time_series = np.expand_dims(time_series, axis=(0, -1))  # (1, time_series_length, 1)
    kernel_functions = np.expand_dims(kernel_functions, axis=1)  # (num_kernel_functions, 1, kernel_length)
    kernel_functions = np.transpose(kernel_functions, (2, 1, 0))  # (kernel_length, 1, num_kernel_functions)

    # 2. Convert to TensorFlow tensors
    time_series_tf = tf.convert_to_tensor(time_series, dtype=tf.float16)
    kernel_functions_tf = tf.convert_to_tensor(kernel_functions, dtype=tf.float16)

    # 3. Convolution in TensorFlow
    convolved_tf = tf.nn.convolution(
        input=time_series_tf,
        filters=kernel_functions_tf,
        padding='VALID',
        strides=1,
    )
    convolved_tf = tf.squeeze(convolved_tf, axis=0)  # Remove batch dim

    # 4. Reshape time_series_tf for subtraction (all in TensorFlow)
    time_series_tf = tf.reshape(time_series_tf, [time_series_tf.shape[1]])  #Now remove the dims for the tf tensor
    starting_index = int(time_series_tf.shape[0] - convolved_tf.shape[0])
    time_series_sliced_tf = time_series_tf[starting_index:] #Slice now in TF

    # 5. Reshape for subtraction (all in TensorFlow)
    # Determine if convolved_tf needs to be reshaped based on time_series_sliced_tf's shape
    if time_series_sliced_tf.shape[0] == convolved_tf.shape[1]:
        time_series_reshaped_tf = tf.reshape(time_series_sliced_tf, [1, -1])
    elif time_series_sliced_tf.shape[0] == convolved_tf.shape[0]:
        time_series_reshaped_tf = tf.reshape(time_series_sliced_tf, [-1, 1])
    else:
        raise ValueError("Arrays are not compatible. time_series must match either the number of rows or columns of filter outputs.")
    
    # 6. Perform subtraction *directly in TensorFlow* (No CuPy conversion anymore!)
    intermediate_result = time_series_reshaped_tf - convolved_tf
    scaled_result = intermediate_result.numpy()
    with tqdm(desc="Processing") as pbar:
        for i in range(scaled_result.shape[1]):
            scaled_result[:,i] /= np.std(scaled_result[:,i])
            pbar.update(1)
            
    return scaled_result

# Example Time Series
length = 5000
sample_rate = 10
time_series = tsa.symmetric_random_walk(length, sample_rate)
resid = conv_subtract_scale(time_series, kernel_functions[0])

Processing: 0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:187: RuntimeWarning: overflow encountered in reduce
Processing: 3746it [00:00, 4036.02it/s]


# Computing Sliding L-Moments

In [31]:
def sliding_lmoments(data, window_size, orders):
    """
    Compute L-moments over a sliding window of a 1D NumPy array using sliding_window_view.
    
    Parameters:
    - data: 1D NumPy array, time series data
    - window_size: int, size of the sliding window
    - orders: int or list of ints, number of L-moments or specific orders
    
    Returns:
    - 2D NumPy array: rows are windows, columns are L-moments
    
    Raises:
    - ValueError: if inputs are invalid
    """
    # Input validation
    if not isinstance(data, np.ndarray) or data.ndim != 1:
        raise ValueError("data must be a 1D NumPy array")
    if not isinstance(window_size, int) or window_size <= 0:
        raise ValueError("window_size must be a positive integer")
    if window_size > len(data):
        raise ValueError("window_size must be <= data length")
    
    # Determine number of L-moments to compute
    if isinstance(orders, int):
        if orders <= 0:
            raise ValueError("orders must be a positive integer")
        max_order = orders
        idx = slice(None)  # Return all computed L-moments
    elif isinstance(orders, list):
        if not orders or not all(isinstance(i, int) and i > 0 for i in orders):
            raise ValueError("orders must be a non-empty list of positive integers")
        max_order = max(orders)
        idx = [o - 1 for o in orders]  # Indices for specific L-moments
    else:
        raise ValueError("orders must be an integer or a list of positive integers")
    
    # Create sliding windows
    windows = sliding_window_view(data, window_size)
    n_windows = windows.shape[0]
    
    # Compute L-moments for each window
    # lm.lmoments doesn't vectorize, so we use a list comprehension
    lmoments = np.array([lm.lmom_ratios(window, max_order) for window in windows])
    
    # Return selected L-moments
    return lmoments[:, idx]

# Example usage
if __name__ == "__main__":
    # Sample data
    data = tsa.symmetric_random_walk(500, 20)
    
    # Compute first 3 L-moments over window size 3
    result1 = sliding_lmoments(data, window_size=50, orders=6)
    #print("First 3 L-moments:\n", result1)
    
    # Compute specific L-moments (L1 and L2) over window size 3
    result2 = sliding_lmoments(data, window_size=50, orders=[1, 2])
    #print("\nL1 and L2 only:\n", result2)

In [43]:
import cupy as cp
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view
import lmoments3 as lm

def sliding_lmoments_gpu(data, window_size, orders):
    """
    Compute L-moments over sliding windows using CuPy for GPU acceleration.
    
    Parameters:
    - data: 1D NumPy or CuPy array, time series data
    - window_size: int, size of the sliding window
    - orders: int or list of ints, number of L-moments or specific orders
    
    Returns:
    - 2D NumPy array: rows are windows, columns are L-moments
    """
    # Convert input to CuPy array if it’s NumPy
    if isinstance(data, np.ndarray):
        data = cp.asarray(data)
    elif not isinstance(data, cp.ndarray):
        raise ValueError("data must be a 1D NumPy or CuPy array")
    if data.ndim != 1:
        raise ValueError("data must be 1D")
    if window_size > len(data):
        raise ValueError("window_size must be <= data length")
    
    # Determine L-moment orders
    if isinstance(orders, int):
        if orders <= 0:
            raise ValueError("orders must be a positive integer")
        max_order = orders
        idx = slice(None)
    elif isinstance(orders, list):
        if not orders or not all(isinstance(i, int) and i > 0 for i in orders):
            raise ValueError("orders must be a non-empty list of positive integers")
        max_order = max(orders)
        idx = [o - 1 for o in orders]
    else:
        raise ValueError("orders must be an integer or a list of positive integers")
    
    # Create sliding windows on GPU
    # CuPy doesn’t have sliding_window_view, so we use NumPy and transfer
    windows_np = sliding_window_view(data.get(), window_size)
    windows = cp.asarray(windows_np)  # Move to GPU
    
    # Compute L-moments (on CPU due to lmoments3 limitation)
    # Transfer each window to CPU, compute, and stack results
    lmoments = np.array([lm.lmom_ratios(cp.asnumpy(window), max_order) 
                         for window in windows])
    
    # Return selected L-moments as NumPy array
    return lmoments[:, idx]

# Example usage
if __name__ == "__main__":
    # Sample data
    data = tsa.symmetric_random_walk(50000, 20)
    
    # Run on GPU
    result = sliding_lmoments_gpu(data, window_size=30, orders=6)
    print("L1 and L2:\n", result)

L1 and L2:
 [[ 8.13333333e-01  5.55632184e-01  2.81990661e-01  1.89668420e-01
  -1.05259196e-01 -6.38999395e-02]
 [ 8.73333333e-01  5.72873563e-01  2.49770695e-01  1.51475197e-01
  -1.26181069e-01 -5.00823147e-02]
 [ 9.26666667e-01  5.95402299e-01  2.17209046e-01  1.00508672e-01
  -1.37891109e-01 -1.84866185e-02]
 ...
 [ 1.61800000e+01  7.79310345e-01  7.49683944e-02 -8.12380016e-03
   3.54666638e-03  2.47459849e-02]
 [ 1.61000000e+01  7.36551724e-01  7.66452648e-02  1.02401760e-02
   1.67735233e-02  3.52556100e-02]
 [ 1.60400000e+01  6.91724138e-01  5.04201681e-02  9.82238470e-03
   3.12071518e-02  4.68395384e-02]]


In [46]:
import cProfile
import pstats

# Profile with cProfile
profiler = cProfile.Profile()
profiler.enable()

result = sliding_lmoments_gpu(data, 30, 6)

profiler.disable()
stats = pstats.Stats(profiler).sort_stats('cumulative')
stats.print_stats()

Exception ignored When destroying _lsprof profiler:
Traceback (most recent call last):
  File "<ipython-input-46-b71cffd9484e>", line 6, in <cell line: 6>
RuntimeError: Cannot install a profile function while another profile function is being installed


         205580931 function calls in 111.815 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000  111.815   55.908 /usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3512(run_code)
        2    0.000    0.000  111.815   55.907 {built-in method builtins.exec}
        1    0.000    0.000  111.815  111.815 <ipython-input-46-b71cffd9484e>:1(<cell line: 8>)
        1    0.008    0.008  111.815  111.815 <ipython-input-43-c914fbf742a7>:6(sliding_lmoments_gpu)
        1    0.775    0.775  111.770  111.770 <ipython-input-43-c914fbf742a7>:49(<listcomp>)
    49971    0.132    0.000  108.872    0.002 /usr/local/lib/python3.10/dist-packages/lmoments3/__init__.py:74(lmom_ratios)
    49971   51.177    0.001  108.740    0.002 /usr/local/lib/python3.10/dist-packages/lmoments3/__init__.py:90(_samlmularge)
 37778076   25.927    0.000   50.494    0.000 /usr/local/lib/python3.10/dist-packages/scip

In [2]:
def find_equal_arrays(array_list):
    """
    Finds and groups NumPy arrays in a list that are equal to each other.

    Args:
        array_list (list of numpy arrays): A list containing NumPy arrays.

    Returns:
        list of lists: A list where each inner list contains the *indices* of arrays
                       in the input `array_list` that are equal to each other.  
                       Arrays are considered equal if all their elements are equal.

                       For example, if array_list = [array1, array2, array3]
                       and array1 and array3 are equal, the function might return:
                       [[0, 2], [1]]  (array1 and array3 are at indices 0 and 2, array2 at 1)

                       or
                       [[1], [0, 2]]

                       The order of the inner lists and the order of indices within the inner
                       lists is not guaranteed.
    """

    equal_groups = []
    remaining_indices = list(range(len(array_list)))  # Indices to process

    while remaining_indices:
        first_index = remaining_indices.pop(0)  # Take the first unprocessed array
        equal_group = [first_index]
        first_array = array_list[first_index]

        # Compare to all remaining unprocessed arrays
        indices_to_remove = []
        for i in remaining_indices:
            other_array = array_list[i]
            if np.array_equal(first_array, other_array):
                equal_group.append(i)
                indices_to_remove.append(i)

        # Remove the matched indices from remaining_indices (in reverse order to avoid shifting issues)
        for i in sorted(indices_to_remove, reverse=True):
            remaining_indices.remove(i)

        equal_groups.append(equal_group)

    return equal_groups


# Example Usage:
array1 = np.array([1, 2, 3])
array2 = np.array([4, 5, 6])
array3 = np.array([1, 2, 3])  # Equal to array1
array4 = np.array([7, 8, 9])
array5 = np.array([4, 5, 6])  # Equal to array2
array6 = np.array([1, 2, 3])  # Equal to array1 and array3

my_list = [array1, array2, array3, array4, array5, array6]
equality_groups = find_equal_arrays(my_list)
print(f"Arrays equal to each other (grouped by indices): {equality_groups}")


array7 = np.array([1, 1, 1])
array8 = np.array([1, 1, 1])
array9 = np.array([2, 2, 2])

my_list2 = [array7, array8, array9]
equality_groups2 = find_equal_arrays(my_list2)
print(f"Arrays equal to each other (grouped by indices) 2: {equality_groups2}")

Arrays equal to each other (grouped by indices): [[0, 2, 5], [1, 4], [3]]
Arrays equal to each other (grouped by indices) 2: [[0, 1], [2]]
